In [24]:
# install.packages("RSQLite")
# install.packages("Hmisc")
library("DBI")
library(Hmisc)
date <- "2018-11-14"

In [25]:
conn <- dbConnect(RSQLite::SQLite(), "stop_times.db")
res <- dbSendQuery(conn, paste0("SELECT arrival_time,\"", date, "\" FROM stop_times WHERE \"", date, "\" IS NOT NULL"))
all_vals <- dbFetch(res)
vals <- all_vals[[date]]

### Some summary statistics

In [26]:
Hmisc::describe(vals)
summary(vals)

 
NULL


Length  Class   Mode 
     0   NULL   NULL 

In [23]:
print(paste0("average delay of buses: ", round(mean(vals), 4), " seconds"))
print(paste0("buses that arrived exactly on time: ", round(mean(vals == 0), 3)*100, "%"))
print(paste0("buses that weren't early but less than a minute late: ", round(mean(vals >= 0 & vals < 60), 3)*100, "%"))
print(paste0("buses that were within 30 seconds of their scheduled time: ", round(mean(abs(vals) < 30), 3)*100, "%"))
print(paste0("buses that were more than 5 minutes late: ", round(mean(vals > 300), 3)*100, "%"))
print(paste0("buses that were more than 5 minutes early: ", round(mean(vals < -300), 3)*100, "%"))

Warning message in mean.default(vals):
“argument is not numeric or logical: returning NA”

[1] "average delay of buses: NA seconds"
[1] "buses that arrived exactly on time: NaN%"
[1] "buses that weren't early but less than a minute late: NaN%"


ERROR: Error in abs(vals): non-numeric argument to mathematical function


In [ ]:
histinfo <- hist(vals, breaks=1000, freq=FALSE, xlab="Bus Delay (sec)", main=paste("Histogram of Bus Delays on", date))

In [ ]:
head(sorted_highest_peaks <- histinfo$density[order(histinfo$density, decreasing = TRUE)])

In [ ]:
hist(vals, breaks=1000, xlim=c(mean(vals)-2*sd(vals), mean(vals)+2*sd(vals)), ylim=c(0,sorted_highest_peaks[2]), freq=FALSE, xlab="Bus Delay (sec)", main=paste("[Zoomed +-2SD] Histogram of Bus Delays on", date))

In [ ]:
hist(vals, breaks=1000, ylim=c(0,sorted_highest_peaks[3]), freq=FALSE, xlab="Bus Delay (sec)", main=paste("[Zoomed in Density] Histogram of Bus Delays on", date))

In [ ]:
hist(vals, breaks=1000, ylim=c(0,sorted_highest_peaks[3]), freq=FALSE, xlab="Bus Delay (sec)", main="Histogram of Bus Delays + Normal Curve Superimposed")
curve(dnorm(x, mean(vals), sd(vals)), add=TRUE, col='RED')

In [ ]:
arrTimes <- as.list(all_vals[['arrival_time']])
arrTimeDates <- sapply(arrTimes, function (time) {paste(date, time)})
arrPOSIX <- as.POSIXct(arrTimeDates, format="%Y-%m-%d %H:%M:%S", origin="1960-01-01")
cols <- rep(1, length(vals))
cols[vals == 0] <- 2
plot(vals ~ arrPOSIX, xlab="Time", ylab="Delay (sec)", main=paste("Delay of buses over Time of Day on", date), las=1, pch=20, col=cols)
axis.POSIXct(1, x=arrPOSIX)

# add vertical lines to show beginning of hour, in case variation was higher at the beginnning of the hour
lines <- as.POSIXct(sapply(seq(0, 24, 1), function (time) {paste(date, time)}), format="%Y-%m-%d %H")
abline(v=lines)

# linear model
fit <- lm(vals ~ arrPOSIX)
abline(fit, col="BLUE", lwd=3)

# technically shouldn't be correlated? need more data to prove (could be sample size causing less variability)
summary(fit)

In [ ]:
res <- dbSendQuery(conn, paste0("SELECT DISTINCT route_id FROM stop_times"))
all_routes <- dbFetch(res)
length(all_routes[['route_id']])

In [ ]:
res <- dbSendQuery(conn, paste0("SELECT route_id, \"", date, "\" FROM stop_times ORDER BY route_id"))
head(all_departures <- dbFetch(res))

In [ ]:
data <- all_departures[[date]]
confint95 <- c(-1.96, 1.96) * sd(data, na.rm=TRUE) + mean(data, na.rm=TRUE)
filter <- all_departures[[date]] < confint95[2] & all_departures[[date]] > confint95[1]
sorted_routes <- order(unique(all_departures[['route_id']][filter]))

# png(filename="all_routes.png", width=10, height=10, units="in", res=800)
histogram(~data[filter] | all_departures[['route_id']][filter],
          xlab="Delay of bus arrivals (seconds)",
          xlim=c(confint95[1],confint95[2]),
          ylim=c(0,5),
          main=paste0("Delay of bus arrivals by bus routes (collected on ", date, ")"),
          breaks=seq(confint95[1]-5, confint95[2]+5,5),
          par.strip.text=list(cex=0.5),
          layout=c(round(length(sorted_routes) / 9, 0) + 1, 9),
          panel=function(x,...) {
              panel.histogram(x,...)
              panel.text(300,4,
                         labels=paste0(
                             'mean: ', round(mean(x), 2), '\nsd: ', round(sd(x), 2), '\nn: ', length(x)),
                             cex=0.3)
          })
# dev.off()